In [1]:
from pymongo import MongoClient
from pymongo import GEOSPHERE
import geopandas as gpd
import pandas as pd
import shapely

In [2]:
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

# Getting dataframes per city

In [3]:
dfsanfran = pd.read_csv('../newdata/dfsanfran.csv')
dfell = pd.read_csv('../newdata/dfellensburg.csv')
dfbro = pd.read_csv('../newdata/dfbrooklyn.csv')

# Getting mongo collections per city

In [4]:
sf = db.get_collection("sanfrancisco")
ell = db.get_collection("ellensburg")
bro = db.get_collection("brooklyn")

# Making geodataframes

In [5]:
geosf = gpd.GeoDataFrame(dfsanfran, geometry=gpd.points_from_xy(dfsanfran.lon, dfsanfran.lat))
geosf.drop("location",axis=1, inplace=True)

geoell = gpd.GeoDataFrame(dfell, geometry=gpd.points_from_xy(dfell.lon, dfell.lat))
geoell.drop("location",axis=1, inplace=True)

geobro = gpd.GeoDataFrame(dfbro, geometry=gpd.points_from_xy(dfbro.lon, dfbro.lat))
geobro.drop("location",axis=1, inplace=True)

# Creating indexes

## San Francisco

In [6]:
geosf['geometry']=geosf['geometry'].apply(lambda x:shapely.geometry.mapping(x))
geosfdic = geosf.to_dict(orient="records")
sf.create_index([("geometry", GEOSPHERE)])
sf.insert_many(geosfdic)

/home/andrestart/anaconda3/envs/iron/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


## Ellensburg

In [7]:
geoell['geometry']=geoell['geometry'].apply(lambda x:shapely.geometry.mapping(x))
geoelldic = geoell.to_dict(orient="records")
ell.create_index([("geometry", GEOSPHERE)])
ell.insert_many(geoelldic)

/home/andrestart/anaconda3/envs/iron/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


## Brooklyn

In [8]:
geobro['geometry']=geobro['geometry'].apply(lambda x:shapely.geometry.mapping(x))
geobrodic = geobro.to_dict(orient="records")
bro.create_index([("geometry", GEOSPHERE)])
bro.insert_many(geobrodic)

/home/andrestart/anaconda3/envs/iron/lib/python3.9/site-packages/geopandas/geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


# Making the Mongo gequeries to  get the distance from the cities' coordinates to the targetted locations

In [9]:
sanfrancisco = {'type': 'Point', 'coordinates':[-122.40559118863916, 37.778480215129015]}
ellensburg =  {'type': 'Point', 'coordinates':[-120.54550769638885, 46.996517823587766]}
brooklyn =  {'type': 'Point', 'coordinates':[40.692147496548735, -73.98495174207916]}

In [10]:
metres = 3000

## San Francisco geoquery

In [11]:
querysf = [{"$geoNear": {
            "near": sanfrancisco['coordinates'],
            "distanceField": "geodist",
            "maxDistance": 3000,
            "distanceMultiplier" :6371,
            "spherical": True}}]
geoloc_sf = list(sf.aggregate(querysf))

In [12]:
geoloc_sf = pd.DataFrame(geoloc_sf)
geoloc_sf.drop(['_id','distance','geometry'], axis=1, inplace=True)

In [13]:
geoloc_sf.to_csv(f"../newdata/dfgeosf.csv", index = False)

## Ellensburg geoquery

In [14]:
queryell = [{"$geoNear": {
            "near": ellensburg['coordinates'],
            "distanceField": "geodist",
            "maxDistance": 3000,
            "distanceMultiplier" :6371,
            "spherical": True}}]
geloc_ell = list(ell.aggregate(queryell))

In [15]:
geloc_ell = pd.DataFrame(geloc_ell)
geloc_ell.drop(['_id','distance','geometry'], axis=1, inplace=True)

In [16]:
geloc_ell.to_csv(f"../newdata/dfgeoell.csv", index = False)

## Brooklyn geoquery

In [17]:
querybro = [{"$geoNear": {
            "near": brooklyn['coordinates'],
            "distanceField": "geodist",
            "maxDistance": 3000,
            "distanceMultiplier" :6371,
            "spherical": True}}]
geoloc_bro = list(bro.aggregate(querybro))

In [18]:
geoloc_bro = pd.DataFrame(geoloc_bro)
geoloc_bro.drop(['_id','distance','geometry'], axis=1, inplace=True)

In [19]:
geoloc_bro['geodist'] = geoloc_bro['geodist']/100000
geoloc_bro

,name,lat,lon,category,geodist
0,MTA Subway - Grand Army Plaza (2/3),40.675184,-73.971031,train_stations,0.150718
1,MTA Subway - 4th Ave/9th St (F/G/R),40.670102,-73.989461,train_stations,0.150718
2,PS 118 - Maurice Sedak School,40.671092,-73.987976,schools,0.150719
3,Berkeley Carroll (Lower School),40.673925,-73.978320,schools,0.150719
4,Berkeley Carroll (Upper School),40.675633,-73.973946,schools,0.150719
...,...,...,...,...,...
91,Starbucks,40.702694,-73.990906,starbucks,0.150752
92,Miami Ad School Brooklyn,40.704596,-73.987520,schools,0.150753
93,St. James-St. Joseph School,40.711844,-73.996865,schools,0.150764
94,MTA Subway - East Broadway (F),40.713973,-73.990125,train_stations,0.150764


In [20]:
geoloc_bro.to_csv(f"../newdata/dfgeobro.csv", index = False)